In [2]:
import RPi.GPIO as GPIO
import smbus

import time
import math

# Broadcom chip-specific pin numbers. These pin numbers follow the lower-level
# numbering system defined by the Raspberry Pi's Broadcom-chip brain.
GPIO.setmode(GPIO.BCM)

GPIO.setwarnings(False)

## Common: abstractions

Abstract common functionality in the little helper classes to avoid code duplication.

### Raspi PCA9685 16-Channel PWM Servo Driver

In [3]:
class PCA9685:
    # Registers/etc.
    __MODE1 = 0x00
    __PRESCALE = 0xFE
    __LED0_ON_L = 0x06
    __LED0_ON_H = 0x07
    __LED0_OFF_L = 0x08
    __LED0_OFF_H = 0x09

    def __init__(self, address=0x40, debug=False):
        """Raspi PCA9685 16-Channel PWM Servo Driver"""
        self.bus = smbus.SMBus(1)
        self.address = address
        self.debug = debug
        self.write(self.__MODE1, 0x00)

    def write(self, reg, value):
        "Writes an 8-bit value to the specified register/address"
        self.bus.write_byte_data(self.address, reg, value)

    def read(self, reg):
        "Read an unsigned byte from the I2C device"
        result = self.bus.read_byte_data(self.address, reg)
        return result

    def set_pwm_freq(self, freq):
        "Sets the PWM frequency"
        prescaleval = 25000000.0    # 25MHz
        prescaleval /= 4096.0       # 12-bit
        prescaleval /= float(freq)
        prescaleval -= 1.0
        prescale = math.floor(prescaleval + 0.5)

        old_mode = self.read(self.__MODE1)
        new_mode = (old_mode & 0x7F) | 0x10        # sleep
        self.write(self.__MODE1, new_mode)        # go to sleep
        self.write(self.__PRESCALE, int(math.floor(prescale)))
        self.write(self.__MODE1, old_mode)
        time.sleep(0.005)
        self.write(self.__MODE1, old_mode | 0x80)

    def set_pwm(self, channel, on, off):
        "Sets a single PWM channel"
        self.write(self.__LED0_ON_L + 4 * channel, on & 0xFF)
        self.write(self.__LED0_ON_H + 4 * channel, on >> 8)
        self.write(self.__LED0_OFF_L + 4 * channel, off & 0xFF)
        self.write(self.__LED0_OFF_H + 4 * channel, off >> 8)

    def set_motor_pwm(self, channel, duty):
        self.set_pwm(channel, 0, duty)

    def set_servo_pulse(self, channel, pulse):
        "Sets the Servo Pulse,The PWM frequency must be 50HZ"
        pulse = pulse * 4096 / 20000  # PWM frequency is 50HZ, the period is 20000us
        self.set_pwm(channel, 0, int(pulse))

## Buzzer: make a noise

In [16]:
BUZZER_PIN = 17

GPIO.setup(BUZZER_PIN, GPIO.OUT)
GPIO.output(BUZZER_PIN, True)
time.sleep(0.25)
GPIO.output(BUZZER_PIN, False)

## Ultrasonic: get distance to obstacle

In [87]:
TRIGGER_PIN = 27
ECHO_PIN = 22
MAX_DISTANCE = 300 # maximum measuring distance, unit cm
TIME_OUT = MAX_DISTANCE * 60
SOUND_SPEED = 340.0 # m/s

GPIO.setup(TRIGGER_PIN, GPIO.OUT)
GPIO.setup(ECHO_PIN, GPIO.IN)

def pulse_in(pin, level, time_out):
    """ Obtains pulse time of a pin under TIME_OUT """
    t0 = time.time()
    while(GPIO.input(pin) != level):
        if((time.time() - t0) > time_out * 0.000001):
            return 0
    t0 = time.time()
    while(GPIO.input(pin) == level):
        if((time.time() - t0) > time_out * 0.000001):
            return 0
    return (time.time() - t0) * 1000000

def get_distance():
    """ Gets the measurement results of ultrasonic module,with unit: cm """
    ATTEMPTS = 5
    distance_cm = [0] * ATTEMPTS
    for i in range(ATTEMPTS):
        # make trigger_pin output 10us HIGH level
        GPIO.output(TRIGGER_PIN, GPIO.HIGH)
        time.sleep(0.00001) # 10us
        # make trigger_pin output LOW level
        GPIO.output(TRIGGER_PIN, GPIO.LOW)
        # read plus time of echo_pin
        pingTime = pulse_in(ECHO_PIN, GPIO.HIGH, TIME_OUT)
        # calculate distance with sound speed
        distance_cm[i] = pingTime * SOUND_SPEED / 2.0 / 10000.0
    distance_cm = sorted(distance_cm)

    print(f"{distance_cm=}")
    
    return int(distance_cm[2])

get_distance()

distance_cm=[0.0, 0.0, 12.260675430297852, 12.268781661987305, 12.293100357055664]


12

## Camera Servo

In [13]:
class DIRECTION:
    HORIZONTAL = 0,
    VERTICAL = 1


def move_servo_pwm(pwm_servo, direction: DIRECTION, angle, error=10):
    angle = int(angle)
    value = int((angle + error) / 0.09)

    if direction == DIRECTION.HORIZONTAL:
        pwm_servo.set_servo_pulse(8, 2500 - value)
    else:
        pwm_servo.set_servo_pulse(9, 500 + value)

def reset(pwm_servo):
    """ Resets to initial position """
    pwm_servo.set_pwm_freq(50)
    pwm_servo.set_servo_pulse(8, 1500)
    pwm_servo.set_servo_pulse(9, 1500)

pwm_servo = PCA9685()

reset(pwm_servo)

time.sleep(0.5)

# move horizontally for given angle
move_servo_pwm(pwm_servo, direction=DIRECTION.HORIZONTAL, angle=100)

time.sleep(1)

# move vertically for given angle
move_servo_pwm(pwm_servo, direction=DIRECTION.VERTICAL, angle=100)

time.sleep(1)

reset(pwm_servo)

## Motor: wheels

In [12]:
import time

def clamp(n, minn, maxn):
    return max(min(maxn, n), minn)

class Wheels:
    """ Provides api to control 4 vehicle wheels. """
    def __init__(self, pwm):
        self.pwm = pwm
        self.pwm.set_pwm_freq(50)

    def duty_range(self, duty1, duty2, duty3, duty4):
        duty1 = clamp(duty1, -4095, 4095)
        duty2 = clamp(duty2, -4095, 4095)
        duty3 = clamp(duty3, -4095, 4095)
        duty4 = clamp(duty4, -4095, 4095)

        return duty1, duty2, duty3, duty4

    def left_upper_wheel(self, duty):
        if duty > 0:
            self.pwm.set_motor_pwm(0, 0)
            self.pwm.set_motor_pwm(1, duty)
        elif duty < 0:
            self.pwm.set_motor_pwm(1, 0)
            self.pwm.set_motor_pwm(0, abs(duty))
        else:
            self.pwm.set_motor_pwm(0, 4095)
            self.pwm.set_motor_pwm(1, 4095)

    def left_lower_wheel(self, duty):
        if duty > 0:
            self.pwm.set_motor_pwm(3, 0)
            self.pwm.set_motor_pwm(2, duty)
        elif duty < 0:
            self.pwm.set_motor_pwm(2, 0)
            self.pwm.set_motor_pwm(3, abs(duty))
        else:
            self.pwm.set_motor_pwm(2, 4095)
            self.pwm.set_motor_pwm(3, 4095)

    def right_upper_wheel(self, duty):
        if duty > 0:
            self.pwm.set_motor_pwm(6, 0)
            self.pwm.set_motor_pwm(7, duty)
        elif duty < 0:
            self.pwm.set_motor_pwm(7, 0)
            self.pwm.set_motor_pwm(6, abs(duty))
        else:
            self.pwm.set_motor_pwm(6, 4095)
            self.pwm.set_motor_pwm(7, 4095)

    def right_lower_wheel(self, duty):
        if duty > 0:
            self.pwm.set_motor_pwm(4, 0)
            self.pwm.set_motor_pwm(5, duty)
        elif duty < 0:
            self.pwm.set_motor_pwm(5, 0)
            self.pwm.set_motor_pwm(4, abs(duty))
        else:
            self.pwm.set_motor_pwm(4, 4095)
            self.pwm.set_motor_pwm(5, 4095)

    def set_motor_model(self, duty1, duty2, duty3, duty4):
        duty1, duty2, duty3, duty4 = self.duty_range(duty1, duty2, duty3, duty4)
        self.left_upper_wheel(duty1)
        self.left_lower_wheel(duty2)
        self.right_upper_wheel(duty3)
        self.right_lower_wheel(duty4)

wheels = Wheels(pwm=PCA9685())

wheels.left_upper_wheel(1000)
time.sleep(2)

wheels.right_lower_wheel(-1000)
time.sleep(2)

wheels.set_motor_model(0, 0, 0, 0)

## Power

In [31]:
ADDRESS = 0x48     # I2C address of the device
ADS7830_CMD = 0x84 # ADS7830 Command, Single-Ended Inputs

def recvADS7830(bus, channel):
    """Select the Command data from the given provided value above"""
    COMMAND_SET = ADS7830_CMD | ((((channel << 2) | (channel >> 1)) & 0x07) << 4)
    bus.write_byte(ADDRESS, COMMAND_SET)
    while True:
        value1 = bus.read_byte(ADDRESS)
        value2 = bus.read_byte(ADDRESS)
        if value1 == value2:
            break
    voltage = value1 / 255.0 * 3.3  # calculate the voltage value
    voltage = round(voltage, 2)
    return voltage

bus = smbus.SMBus(1) # get I2C bus

for _ in range(3):
    aa = bus.read_byte_data(ADDRESS, 0xf4)
    index = "PCF8591" if aa < 150 else "ADS7830"

print(f"{index=}")

if index == "ADS7830":
    value = recvADS7830(bus, channel = 2)
    adc_power = value * 3
    adc_power = round(adc_power, 2)
    print(f"{adc_power=}V")


bus.close()

index='ADS7830'
adc_power=7.74V
